In [2]:
#%pip install pandas pyarrow
import pandas as pd

# Load the Parquet file
articles = pd.read_parquet('articles.parquet', engine='pyarrow')
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('train/history.parquet', engine='pyarrow')

# bert = pd.read_parquet('bert_base_multilingual_cased.parquet', engine='pyarrow')
# display = bert['google-bert/bert-base-multilingual-cased']

# Test wikipedia entity embeddings

In [ ]:
with open("../../../../glove.840B.300d.txt", "r", encoding="utf8") as f:
    lines = f.readlines()
    print(lines[0]) 

In [ ]:
entities = []
nr_articles_no_entities = 0
for ner_clusters in articles["ner_clusters"]:
    if len(ner_clusters) == 0:
        nr_articles_no_entities += 1
    for entity in ner_clusters:
        entities.append(entity)
unique_entities = set(entities)
print(f"Number of unique entities: {len(unique_entities)}")
print(f"Percentage of articles without entities: {(nr_articles_no_entities / len(articles)) * 100}%")

In [ ]:
found = 0
found_embeddings = []
for line in lines[1:]:
    entity = line.split()[0]
    if entity in unique_entities:
        found += 1
        found_embeddings.append(entity)
print(f"Percentage of entities found in the wikipedia embeddings: {(found/len(unique_entities)  * 100)}%")    

In [ ]:
# write to file
with open("../entity/entities2id.txt", "w", encoding="utf8") as f:
    for idx,entity in enumerate(set(found_embeddings)):
        f.write(str(idx+1) +"\t" + entity + "\n")
print(idx)

In [1]:
# test file
with open('../entity/entities2id.txt', encoding="utf8") as f:
    lines = f.readlines()

Entity2Index = {}
Index2Entity = {}
print(len(lines))
for line in lines:
    index, word = line.strip('\n').split('\t')
    if word in Entity2Index or index in Index2Entity:
        raise Exception("Duplicate entity")
    Entity2Index[word] = int(index)
    Index2Entity[int(index)] = word
len(Entity2Index)

6742


6742

In [ ]:
Entity2Index

In [ ]:
len(unique_entities)

In [7]:
for idx, article in articles.iterrows():
    print(article)
    break

(0, article_id                                                      3037230
title                        Ishockey-spiller: Jeg troede jeg skulle dø
subtitle              ISHOCKEY: Ishockey-spilleren Sebastian Harts h...
last_modified_time                                  2023-06-29 06:20:57
premium                                                           False
body                  Ambitionerne om at komme til USA og spille ish...
published_time                                      2003-08-28 08:55:00
image_ids                                                          None
article_type                                            article_default
url                   https://ekstrabladet.dk/sport/anden_sport/isho...
ner_clusters                                                         []
entity_groups                                                        []
topics                [Kriminalitet, Kendt, Sport, Katastrofe, Mindr...
category                                                    

In [8]:
doc2entitiesid = {}
for idx, article in articles.iterrows():
    id = article["article_id"]
    doc_entities = article["ner_clusters"]
    for doc_entity in doc_entities:
        if doc_entity not in Entity2Index:
            continue
        if id not in doc2entitiesid:
            doc2entitiesid[id] = []
        doc2entitiesid[id].append(Entity2Index[doc_entity])
            
doc2entitiesid

{3044020: [6594],
 4081857: [6157, 5044],
 4122820: [4763],
 4288610: [3942],
 4298451: [2722],
 4437767: [5370],
 4679777: [829],
 5996581: [901],
 6018488: [1383,
  6412,
  2947,
  4002,
  4802,
  4802,
  4802,
  2117,
  2969,
  4816,
  6157,
  6396,
  6396,
  6396],
 6288963: [1756, 2402],
 6512726: [464,
  5247,
  1768,
  2211,
  6125,
  159,
  4404,
  5935,
  1069,
  2945,
  2647,
  6511,
  1854,
  2148,
  182,
  1862,
  2834,
  1948,
  4270],
 6571460: [5370,
  2813,
  5405,
  1807,
  1789,
  1789,
  5478,
  4002,
  5841,
  5281,
  242,
  4757,
  3640,
  6042,
  2175,
  4357,
  6157,
  2654,
  6649,
  2011,
  4387,
  3330,
  6149,
  152,
  2008],
 6679300: [3109,
  1162,
  361,
  139,
  6410,
  3740,
  5041,
  1397,
  5068,
  785,
  6030,
  2707,
  4968,
  3109,
  1162,
  139,
  6410,
  3740,
  5041,
  1397,
  5068,
  785,
  6030,
  2707,
  4968],
 6739176: [4002, 1575, 1974, 1974, 1974, 1974, 813, 6463, 6456, 2297, 793],
 6887342: [4002, 311, 6611, 4533, 3712, 1290],
 6930154: [

In [9]:
# write
with open("../entity/doc2entitiesid.txt", "w", encoding="utf8") as f:
    for doc_id, entities in doc2entitiesid.items():
        f.write(str(doc_id) + "\t" + " ".join([str(entity) for entity in entities]) + "\n")

# test
test_dict = {}
with open('../entity/doc2entitiesid.txt', encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        doc_id, entities = line.strip('\n').split('\t')
        entities = entities.split()
        test_dict[doc_id] = [int(entity) for entity in entities]
test_dict         

{'3044020': [6594],
 '4081857': [6157, 5044],
 '4122820': [4763],
 '4288610': [3942],
 '4298451': [2722],
 '4437767': [5370],
 '4679777': [829],
 '5996581': [901],
 '6018488': [1383,
  6412,
  2947,
  4002,
  4802,
  4802,
  4802,
  2117,
  2969,
  4816,
  6157,
  6396,
  6396,
  6396],
 '6288963': [1756, 2402],
 '6512726': [464,
  5247,
  1768,
  2211,
  6125,
  159,
  4404,
  5935,
  1069,
  2945,
  2647,
  6511,
  1854,
  2148,
  182,
  1862,
  2834,
  1948,
  4270],
 '6571460': [5370,
  2813,
  5405,
  1807,
  1789,
  1789,
  5478,
  4002,
  5841,
  5281,
  242,
  4757,
  3640,
  6042,
  2175,
  4357,
  6157,
  2654,
  6649,
  2011,
  4387,
  3330,
  6149,
  152,
  2008],
 '6679300': [3109,
  1162,
  361,
  139,
  6410,
  3740,
  5041,
  1397,
  5068,
  785,
  6030,
  2707,
  4968,
  3109,
  1162,
  139,
  6410,
  3740,
  5041,
  1397,
  5068,
  785,
  6030,
  2707,
  4968],
 '6739176': [4002, 1575, 1974, 1974, 1974, 1974, 813, 6463, 6456, 2297, 793],
 '6887342': [4002, 311, 6611, 

False

## Create mergeimps.tsv (popularity)
The "mergeimps.tsv'' file needs to store the #click and #impression of each news with time information for building popularity signals. Specifically, we first need to divide the time into several buckets in units of one hour. Thus, since the MIND dataset contains user data in 6 weeks, the time periods can be divided into 2476 buckets. Then, we count the #impressions and #clicks of a news in each bucket and store the results in each line of ``mergeimps.tsv'', where the format is :

[News_ID] TAB [Bucket_ID] TAB #Cliks TAB #Impressions.
 
In MIND, since we did not obtain the click information for test data, which can substantially affect the estimation of popularity signals. Thus, I advise that you can use the validation dataset of MIND for evaluation.

In [ ]:
behaviors[:10]

In [ ]:
# anchor = trans2tsp('05/08/2023 11:59:59 PM')
# def parse_time_bucket(date):
#     tsp = trans2tsp(date)
#     tsp = tsp - anchor
#     tsp = tsp//(600 * 2.2)
#     return tsp


# Assuming 'impression_time' is already in datetime format, if not convert it
behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

# Set the start date of your data collection period
start_date = pd.to_datetime('2023-05-18')

# Calculate the hour buckets since the start of the period
behaviors['bucket_id'] = ((behaviors['impression_time'] - start_date).dt.total_seconds() // (600)).astype(int)

# Check if 'article_ids_inview' is already in the desired format
if not behaviors['article_ids_inview'].str.contains('N').any():
    # For each line in the 'article_ids_inview' column, add 'N' upfront
    # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

# Same for article_ids_clicked
if not behaviors['article_ids_clicked'].str.contains('N').any():
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].head()

In [ ]:
behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].sort_values('bucket_id')

In [ ]:
double_dict = {}

for i, row in behaviors.iterrows():

    for article_id in row["article_ids_inview"].split(' '):
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["impressions"] += 1
            
    for article_id in row["article_ids_clicked"].split(' '):
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["clicks"] += 1
        
mergeimps = pd.DataFrame(double_dict.values())

In [ ]:
# mergeimps.to_excel("mergeimps.xlsx", index=False)
mergeimps.to_csv("../popularity/mergeimps.tsv", sep="\t", index=False, header=False)

In [ ]:
# tests
test = mergeimps.groupby("article_id").agg({"clicks": "sum", "impressions": "sum", "bucket_id" :["nunique", "count"]}).reset_index()
test

## Create V21UrlDocs22_title_json.tsv (necessary?)
For implementing our method based on MIND, the "V21UrlDocs22_title_json.tsv'' needs to store all news in the training/val/test set. Each line stores the information of a single news article, where format is based on 

[News_ID] TAB [Category] TAB [Subcategory] TAB [Title] TAB [Url] TAB [Abstract] TAB [Body].
 
You can modify my codes to avoid the load of some unnecessary news data (e.g., Url, Abstract, and Body)

## Create train/val/test.tsv

In [ ]:
def process_ids(row):
    clicked_ids = set(row['article_ids_clicked'].split())
    inview_ids = row['article_ids_inview'].split()
    processed_ids = [id for id in inview_ids if id not in clicked_ids]
    return ' '.join(processed_ids)

# Convert 'impression_time' to string
behaviors['impression_time'] = behaviors['impression_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not behaviors['impression_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

    # Format 'impression_time' in the desired format
    behaviors['impression_time'] = behaviors['impression_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

# Add 'U' before every 'user_id' if it doesn't already start with 'U'
behaviors['user_id'] = behaviors['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

# Convert 'article_ids_inview' to string
behaviors['article_ids_inview'] = behaviors['article_ids_inview'].astype(str)

# Check if 'article_ids_inview' is already in the desired format
if not behaviors['article_ids_inview'].str.contains('N').any():
    # For each line in the 'article_ids_inview' column, add 'N' upfront
    # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x))
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '').str.replace('\n', '')

# Same for article_ids_clicked
if not behaviors['article_ids_clicked'].str.contains('N').any():
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x) )
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

# Select the required columns
train_file = behaviors[['user_id', 'impression_time', 'article_ids_clicked', 'article_ids_inview']]

# Write to a new .tsv file without index
train_file.to_csv('../train.tsv', sep='\t', index=False, header=True)

# Read the file and print the head
df = pd.read_csv('../train.tsv', sep='\t')
df['article_ids_inview'] = df['article_ids_inview'].str.replace('"', '')
df['article_ids_inview'] = df.apply(process_ids, axis=1)

df['tot_clicks'] = df.apply(lambda row: (row['article_ids_clicked'], row['impression_time']),axis=1)
df['positive'] = df['article_ids_clicked']
df['negatives'] = df['article_ids_inview'].to_list()
df['bucket'] = df['impression_time']

df_grouped = df.groupby('user_id').agg({'tot_clicks': lambda x: x.tolist()}).reset_index()

df = df.drop('tot_clicks', axis=1).merge(df_grouped[['user_id', 'tot_clicks']], on='user_id', how='left')

docs = df[['user_id', 'tot_clicks', 'positive', 'negatives', 'bucket']]


# Write the DataFrame back to the .tsv file
train_split = docs[:int(len(docs)*0.8)]
val_split = docs[int(len(docs)*0.8) : int(len(docs)*0.9)]
test_split = docs[int(len(docs)*0.9) : ]

train_split.to_csv('../train.tsv', sep='\t', index=False, header=False)
val_split.to_csv('../val.tsv', sep='\t', index=False, header=False)
test_split.to_csv('../test.tsv', sep='\t', index=False, header=False)



In [ ]:
docs

## Create docs.tsv

In [ ]:

# Select the required columns
articles['body'] = articles['body'].str.replace('\n', ' ')
articles['title'] = articles['title'].str.replace('\n', ' ')
articles['body'] = articles['body'].str.replace('\t', ' ')
# articles['body'] = articles['body'].fillna('')
# articles['category_str'] = articles['category_str'].fillna('')
articles['topics'] = articles['topics'].astype(str).str.replace('\n', ' ')
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))
docs_file = articles[['article_id', 'title', 'category_str', 'subcategory', 'body', 'topics']]

# Write to a new .tsv file without index
docs_file.to_csv('../docs.tsv', sep='\t', index=False, header=False)

## Create docs_pub_time.tsv

In [ ]:
articles['article_id'] = articles['article_id'].apply(lambda x: 'N' + str(x) if not str(x).startswith('N') else str(x))

# Convert 'impression_time' to string
articles['published_time'] = articles['published_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not articles['published_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    articles['published_time'] = pd.to_datetime(articles['published_time'])

    # Format 'impression_time' in the desired format
    articles['published_time'] = articles['published_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
pubdocs_file = articles[['article_id', 'published_time']]

# Write to a new .tsv file without index
pubdocs_file.to_csv('../popularity/docs_pub_time.tsv', sep='\t', index=False, header=False)

## Create entities